In [ ]:

import simple_train

transform=simple_train.get_transform(train=True)
transform

In [ ]:

dataset = simple_train.PennFudanDataset('PennFudanPed', transform)
dataset[0]


In [53]:



#backbone.body.0.0.weight


#model = simple_train.get_fasterrcnn_model_instance_segmentation(2)

model = simple_train.get_fasterrcnn_model(input_channels=6, classes=2)
print(model)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [34]:
for name, parameter in model.named_parameters():
    print(f"{name}, {parameter.size()}")


backbone.body.0.0.weight, torch.Size([16, 3, 3, 3])
backbone.body.1.block.0.0.weight, torch.Size([16, 1, 3, 3])
backbone.body.1.block.1.0.weight, torch.Size([16, 16, 1, 1])
backbone.body.2.block.0.0.weight, torch.Size([64, 16, 1, 1])
backbone.body.2.block.1.0.weight, torch.Size([64, 1, 3, 3])
backbone.body.2.block.2.0.weight, torch.Size([24, 64, 1, 1])
backbone.body.3.block.0.0.weight, torch.Size([72, 24, 1, 1])
backbone.body.3.block.1.0.weight, torch.Size([72, 1, 3, 3])
backbone.body.3.block.2.0.weight, torch.Size([24, 72, 1, 1])
backbone.body.4.block.0.0.weight, torch.Size([72, 24, 1, 1])
backbone.body.4.block.1.0.weight, torch.Size([72, 1, 5, 5])
backbone.body.4.block.2.fc1.weight, torch.Size([24, 72, 1, 1])
backbone.body.4.block.2.fc1.bias, torch.Size([24])
backbone.body.4.block.2.fc2.weight, torch.Size([72, 24, 1, 1])
backbone.body.4.block.2.fc2.bias, torch.Size([72])
backbone.body.4.block.3.0.weight, torch.Size([40, 72, 1, 1])
backbone.body.5.block.0.0.weight, torch.Size([120, 40

In [52]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params += param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


count_parameters(model)


+--------------------------------------------+------------+
|                  Modules                   | Parameters |
+--------------------------------------------+------------+
|         backbone.body.conv1.weight         |    9408    |
|    backbone.body.layer2.0.conv1.weight     |   32768    |
|    backbone.body.layer2.0.conv2.weight     |   147456   |
|    backbone.body.layer2.0.conv3.weight     |   65536    |
| backbone.body.layer2.0.downsample.0.weight |   131072   |
|    backbone.body.layer2.1.conv1.weight     |   65536    |
|    backbone.body.layer2.1.conv2.weight     |   147456   |
|    backbone.body.layer2.1.conv3.weight     |   65536    |
|    backbone.body.layer2.2.conv1.weight     |   65536    |
|    backbone.body.layer2.2.conv2.weight     |   147456   |
|    backbone.body.layer2.2.conv3.weight     |   65536    |
|    backbone.body.layer2.3.conv1.weight     |   65536    |
|    backbone.body.layer2.3.conv2.weight     |   147456   |
|    backbone.body.layer2.3.conv3.weight

41086169

In [ ]:
import torch
#for name, layer in model.named_parameters():
#    print(f"{name}, {layer.size()}")
#    model[name] = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(
#        2, 2), padding=(3, 3), bias=False)
model.backbone.body.conv1 = torch.nn.Conv2d(4, 64, kernel_size=7,
                        stride=2, padding=3, bias=False)

print(model)


In [22]:
import cv2 # opencv
import os
import xml.etree.ElementTree as ET  # elementpath
from dense_optical_flow import DenseOpticalFlow
import shutil

def get_categories(xml_files):
    """Generate category name to id mapping from a list of xml files.
    
    Arguments:
        xml_files {list} -- A list of xml file paths.
    
    Returns:
        dict -- category name to id mapping.
    """
    classes_names = []
    for xml_file in xml_files:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall("object"):
            classes_names.append(member[0].text)
    classes_names = list(set(classes_names))
    classes_names.sort()
    return {name: i for i, name in enumerate(classes_names)}


def convert(xml_files, json_file):
    json_dict = {"images": [], "type": "instances",
                 "annotations": [], "categories": []}
    if PRE_DEFINE_CATEGORIES is not None:
        categories = PRE_DEFINE_CATEGORIES
    else:
        categories = get_categories(xml_files)
    bnd_id = START_BOUNDING_BOX_ID
    for xml_file in xml_files:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        path = get(root, "path")
        if len(path) == 1:
            filename = os.path.basename(path[0].text)
        elif len(path) == 0:
            filename = get_and_check(root, "filename", 1).text
        else:
            raise ValueError("%d paths found in %s" % (len(path), xml_file))
        ## The filename must be a number
        image_id = get_filename_as_int(filename)
        size = get_and_check(root, "size", 1)
        width = int(get_and_check(size, "width", 1).text)
        height = int(get_and_check(size, "height", 1).text)
        image = {
            "file_name": filename,
            "height": height,
            "width": width,
            "id": image_id,
        }
        json_dict["images"].append(image)
        ## Currently we do not support segmentation.
        #  segmented = get_and_check(root, 'segmented', 1).text
        #  assert segmented == '0'
        for obj in get(root, "object"):
            category = get_and_check(obj, "name", 1).text
            if category not in categories:
                new_id = len(categories)
                categories[category] = new_id
            category_id = categories[category]
            bndbox = get_and_check(obj, "bndbox", 1)
            xmin = int(get_and_check(bndbox, "xmin", 1).text) - 1
            ymin = int(get_and_check(bndbox, "ymin", 1).text) - 1
            xmax = int(get_and_check(bndbox, "xmax", 1).text)
            ymax = int(get_and_check(bndbox, "ymax", 1).text)
            assert xmax > xmin
            assert ymax > ymin
            o_width = abs(xmax - xmin)
            o_height = abs(ymax - ymin)
            ann = {
                "area": o_width * o_height,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [xmin, ymin, o_width, o_height],
                "category_id": category_id,
                "id": bnd_id,
                "ignore": 0,
                "segmentation": [],
            }
            json_dict["annotations"].append(ann)
            bnd_id = bnd_id + 1

    for cate, cid in categories.items():
        cat = {"supercategory": "none", "id": cid, "name": cate}
        json_dict["categories"].append(cat)

    os.makedirs(os.path.dirname(json_file), exist_ok=True)
    json_fp = open(json_file, "w")
    json_str = json.dumps(json_dict)
    json_fp.write(json_str)
    json_fp.close()


def get_or_create_dir(path,dir):
    full_path = os.path.join(path, dir)
    if not os.path.exists(full_path):
        os.mkdir(full_path)
    return full_path

input_dir = '../data/PESMOD/'
output_dir = 'PESMOD'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

dirs= ['train','test']

for tldir in dirs:
    print(tldir)
    input_top_path = os.path.join(input_dir,tldir)
    output_top_path = get_or_create_dir(output_dir, tldir)
    output_annotations_path = get_or_create_dir(output_top_path, 'annotations')
    output_images_path = get_or_create_dir(output_top_path, 'images')
    output_optical_flow_path = get_or_create_dir(output_top_path, 'optical_flow')

    file_number = 0

    for video_name in os.listdir(input_top_path):
        
        video_path = os.path.join(input_top_path, video_name)
        
        
        input_annotations_path = os.path.join(video_path, 'annotations')
        annotations = list(sorted(os.listdir(input_annotations_path)))
        full_path_annotations = map(lambda annotation: os.path.join(
            input_annotations_path, annotation), annotations)
        print(get_categories(full_path_annotations))

        input_images_path = os.path.join(video_path, 'images')
        images = list(sorted(os.listdir(input_images_path)))
            
        dof=None
        for image_filename in images:
            image_full_name = os.path.join(
                input_images_path, image_filename)
            annotation_full_name = os.path.join(
                input_annotations_path, f"{image_filename.split('.')[0]}.xml")

            image = cv2.imread(image_full_name)
            if not dof:
                shape = image.shape
                dof=DenseOpticalFlow(shape[1],shape[0])
                grey_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                dof.process_grey_frame(grey_frame)
                next
            grey_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            optical_flow_image = dof.process_grey_frame(grey_frame)

            file_basename = f"{file_number:06}"
            output_optical_flow_full_name = os.path.join(
                output_optical_flow_path, f"{file_basename}.jpg")
            cv2.imwrite(output_optical_flow_full_name, optical_flow_image)

            output_image_full_name = os.path.join(
                output_images_path, f"{file_basename}.jpg"
            )
            shutil.copyfile(image_full_name, output_image_full_name)

            output_annotations_full_name = os.path.join(
                output_annotations_path, f"{file_basename}.xml"
            )
            shutil.copyfile(annotation_full_name, output_annotations_full_name)

            file_number += 1












train
{'motion': 0}
dense_optical_flow init(w,h): (1920, 1080)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1080, 1920)
dense_optical_flow shape: (1080, 1920, 3)
(1

FileNotFoundError: [Errno 2] No such file or directory: '../data/PESMOD/test/Annotation/annotations'